In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

from datetime import datetime, timedelta

In [3]:
# import data from MTA
# set intial date, set weeks to pull data
string_date = '170527'
weeks = 4

pull_date = datetime.strptime(string_date, '%y%m%d')
df = pd.read_csv(
    'http://web.mta.info/developers/data/nyct/turnstile/turnstile_' + string_date + '.txt')

for i in range(weeks - 1):
    pull_date = pull_date - timedelta(days=7)
    string_date = datetime.strftime(pull_date, '%y%m%d')
    df = df.append(pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_' + string_date + '.txt'),
                   ignore_index=True)

df.rename(columns=lambda x: x.rstrip(), inplace=True)

In [50]:
# build timestamp fields
# get rid of unnecessary fields
df['timestamp'] = df.apply(lambda x: datetime.strptime(
    x['DATE'] + ' ' + x['TIME'], '%m/%d/%Y %H:%M:%S'), axis=1)
del df['LINENAME']
del df['DIVISION']
del df['DATE']
del df['TIME']

In [52]:
# This is for binning the data into four hour increments
# Copy the original dataframe for ease of use
df_timestamps = df.copy()

# set bins size (hour increments)
bin_size = 4

# create bins (in hours) int(x)*x gets closest 4 hours
df_timestamps['hours'] = df_timestamps.apply(lambda x:
                                             int(((x['timestamp']).timestamp()
                                                  / 3600) / bin_size) * bin_size,
                                             axis=1)
# return bins to datetime format
df_timestamps['bin'] = df_timestamps.apply(lambda x: datetime.fromtimestamp(x['hours'] * 3600),
                                           axis=1)

In [53]:
# group by the bins - take min of entries/exits
# this should give us the first value for that time period, assuming it is increasing
# sort the records & take the difference of the bins - we want this to be 4, but sometimes it's not!
df_grouped_bin = df_timestamps.groupby(
    ['C/A', 'UNIT', 'SCP', 'STATION', 'bin']).agg({'ENTRIES': 'min', 'EXITS': 'min'})
df_grouped_bin.reset_index(inplace=True)
df_grouped_bin.sort_values(
    ['C/A', 'UNIT', 'SCP', 'STATION', 'bin'], inplace=True)
df_grouped_bin['diff'] = df_grouped_bin.bin.diff()

In [54]:
# For the next couple cells,
# create all of the different possible bins
string_date = '170528'

# create starting point
first_datetime = datetime.strptime(
    string_date + ' 00:00:00', '%y%m%d %H:%M:%S')

# interate through all of the 4 hour possibilities (time bins)
bin_dict = defaultdict(list)
for i in range(0, 42 * weeks):
    bin_dict['bin'].append(first_datetime - timedelta(hours=4 * i))

# create bin dataframe
bin_df = pd.DataFrame(bin_dict)

In [55]:
# set df2 to all the possible combinations of the key
df2 = df[['C/A', 'UNIT', 'SCP', 'STATION']].copy()
df2.drop_duplicates(inplace=True)
df2.reset_index(inplace=True)

In [56]:
#create a df of all possible key/bin combinations
all_df = df2.assign(foo=1).merge(bin_df.assign(foo=1)).drop('foo', 1)


In [57]:
# left join the dataframe with all combinations with data
# if we're missing values for a bin, they will be null
new_df = pd.merge(all_df, df_grouped_bin,
                  how='left',
                  left_on=['C/A', 'UNIT', 'SCP', 'STATION', 'bin'],
                  right_on=['C/A', 'UNIT', 'SCP', 'STATION', 'bin'])


del new_df['diff']
new_df.sort_values(['C/A', 'UNIT', 'SCP', 'STATION', 'bin'], inplace=True)
new_df.reset_index(inplace=True)

In [58]:
# function to get the median between two values
def get_median(prev_val, next_val):
    try:
        return(prev_val + next_val) / 2
    except:
        return(0)

In [59]:
# calculate the median & input if an entry is missing
# this will not work if two consecutive entries are blank
# helps a bit

# get next and last entry & exit to calculate
new_df['previous_ent'] = new_df['ENTRIES'].shift(1)
new_df['next_ent'] = new_df['ENTRIES'].shift(-1)
new_df['previous_exit'] = new_df['EXITS'].shift(1)
new_df['next_exit'] = new_df['EXITS'].shift(-1)

# actually calculate the medians
new_df['calc_ent'] = new_df.apply(lambda x: get_median(x['previous_ent'], x['next_ent'])
                                  if np.isnan(x['ENTRIES'])
                                  else x['ENTRIES'],
                                  axis=1)
new_df['calc_exit'] = new_df.apply(lambda x: get_median(x['previous_exit'], x['next_exit'])
                                   if np.isnan(x['EXITS'])
                                   else x['EXITS'],
                                   axis=1)
# delete unnecessary rows used for calculation
del new_df['ENTRIES']
del new_df['EXITS']
del new_df['previous_ent']
del new_df['next_ent']
del new_df['previous_exit']
del new_df['next_exit']

In [60]:
# get differences between each bin
new_df['Entry_Count'] = new_df.calc_ent.diff(-1)
new_df['Exit_Count'] = new_df.calc_exit.diff(-1)

# deal with negative values - sometimes its decrementing, so this accounts for that
new_df['Entry_Count'] = new_df.apply(lambda x: -x['Entry_Count']
                                     if x['Entry_Count'] < 0
                                     else x['Entry_Count'],
                                     axis=1)

new_df['Exit_Count'] = new_df.apply(lambda x: -x['Exit_Count']
                                    if x['Exit_Count'] < 0
                                    else x['Exit_Count'],
                                    axis=1)

# if a value is too big to make sense, just make it zero
max_entry_count = 5000

new_df['Entry_Count'] = new_df.apply(lambda x: 0
                                     if x['Entry_Count'] > max_entry_count
                                     else x['Entry_Count'],
                                     axis=1)

new_df['Exit_Count'] = new_df.apply(lambda x: 0
                                    if x['Exit_Count'] > max_entry_count
                                    else x['Exit_Count'],
                                    axis=1)

In [61]:
# if there are still N/A values, make them zero
# there are not many, and they probably aren't worth recovering
new_df['Entry_Count'].fillna(0, inplace=True)
new_df['Exit_Count'].fillna(0, inplace=True)

In [62]:
# group by station and bin to determine the total traffic for each station/time period
df_st_bin = new_df.groupby(["STATION", "bin"]).agg(
    {'Entry_Count': 'sum', 'Exit_Count': 'sum'})
df_st_bin.reset_index(inplace=True)

# total entries and exits
df_st_bin['Total_Traffic'] = df_st_bin.apply(
    lambda x: x['Entry_Count'] + x['Exit_Count'], axis=1)

In [63]:
# function to make cleanly named bin size
def convert_to_range(first, second):
    try:
        return(datetime.strftime(first, "%H:%M") + " to " + datetime.strftime(second, "%H:%M"))
    except:
        return(float('NaN'))


# make a clean dataframe to be output to .CSV
# this will be used in the visualization notebooks
output_df = df_st_bin.copy()

output_df['Day'] = output_df.apply(
    lambda x: datetime.strftime(x['bin'], "%A"), axis=1)
output_df['next_bin'] = output_df['bin'].shift(-1)
output_df['Time_Range'] = output_df.apply(
    lambda x: convert_to_range(x['bin'], x['next_bin']), axis=1)
output_df['DT'] = output_df.apply(
    lambda x: datetime.strftime(x['bin'], '%m/%d/%Y'), axis=1)

# get rid of unnecessary columns
del output_df['bin']
del output_df['next_bin']
del output_df['Entry_Count']
del output_df['Exit_Count']

In [64]:
# output this to CSV for blog visualizations
output_df.to_csv('data/station_traffic.csv')

In [ ]:
# build the output_DF based on mean for each time period
# before, we had a record for every time period combination
# this makes the data weekly
output_df = output_df.groupby(['STATION', 'Day', 'Time_Range'])[
    'Total_Traffic'].mean().reset_index()

In [5]:
# output this to CSV for project visualizations
df.to_csv('data/average_station_traffic.csv')